## Determine Optimal Portfolio Allocation Using Monte Carlo

In [52]:
import datetime as dt
import yfinance as yf
import numpy as np
import pandas as pd
from functools import reduce
from random import randint

# 10 Companies to Design Portfolio Around
tickers = ["coke", 'googl', 'hd', 'ford', 'smg', 'jnj', 'ag']
start = "2000-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

[*********************100%***********************]  7 of 7 completed


### Initial Parameters

In [62]:
iterations = 10000

In [54]:
# Set Date as Index
df_stock_data = df_portfolio.set_index("Date").copy(deep=True)
df_stock_data = df_stock_data["Adj Close"].dropna()

companies = df_stock_data.columns
company_returns = []

for company in companies:
    df_stock_data[f"{company}".lower()] = np.log(df_stock_data[company] / df_stock_data[company].shift(1))
    company_returns.append(f"{company}".lower())

df_stock_data.dropna(inplace=True)

df_returns = df_stock_data[company_returns].copy(deep=True)
df_returns

,ag,coke,ford,googl,hd,jnj,smg
Date,,,,,,,
2006-11-29,0.005115,-0.003918,0.006486,-0.009958,0.014727,0.003480,0.001213
2006-11-30,0.005089,-0.008675,-0.019587,0.000330,0.009261,-0.004390,-0.000607
2006-12-01,-0.033552,-0.021617,-0.035798,-0.008306,0.025995,0.000910,0.000202
2006-12-04,0.020780,0.027460,0.013575,0.008388,0.005885,0.004688,0.003833
2006-12-05,-0.031334,0.011275,-0.018141,0.004425,0.004327,-0.001661,0.001609
...,...,...,...,...,...,...,...
2019-12-24,0.062979,0.005742,-0.020001,-0.004601,0.006638,-0.003489,0.000845
2019-12-26,0.016584,0.021414,0.039609,0.013329,0.000680,-0.000685,-0.000188
2019-12-27,-0.022454,0.003445,-0.039609,-0.005763,-0.003857,-0.000549,0.001500


### Calculate Mean Returns

In [55]:
# Calculating Mean Returns
ag_mean = df_returns["ag"].mean() * 252
coke_mean = df_returns["coke"].mean() * 252
ford_mean = df_returns["ford"].mean() * 252
googl_mean = df_returns["googl"].mean() * 252
hd_mean = df_returns["hd"].mean() * 252
jnj_mean = df_returns["jnj"].mean() * 252
smg_mean = df_returns["smg"].mean() * 252

return_means = [ag_mean, coke_mean, ford_mean, googl_mean, hd_mean, jnj_mean, smg_mean]

i = 0
new_df = {}
new_df_list = []
for ticker in tickers:
    new_df[ticker] = return_means[i]
    i += 1
new_df_list.append(new_df)

df_returns_mean = pd.DataFrame(new_df_list)
df_returns_mean

,coke,googl,hd,ford,smg,jnj,ag
0,0.087624,0.127026,-0.117682,0.129958,0.161476,0.090385,0.095211


### Calculate Risk

In [56]:
# Calculating Covariance Matrix
df_covariance_matrix = df_returns.cov() * 252

### Run Monte Carlo

In [63]:
risk_free_rate = .035

df_final = pd.DataFrame(columns=tickers)

for iteration in range(iterations):
    # Randomly Assign Allocation
    new_df = {}
    new_df_list = []

    for ticker in tickers:
        new_df[ticker] = randint(0,100)
    new_df_list.append(new_df)

    allocation = pd.DataFrame(new_df_list)
    allocation_sum = allocation.sum(axis=1)
    for ticker in tickers:
        allocation[ticker] = allocation[ticker].apply(lambda x: x/allocation_sum)

    expected_return = (df_returns_mean * allocation).sum().sum()
    portfolio_risk = np.sqrt(reduce(np.dot, [allocation, df_covariance_matrix, allocation.T]))

    allocation["expected_return"] = expected_return
    allocation["portfolio_risk"] = portfolio_risk
    allocation["sharp_ratio"] = (expected_return - risk_free_rate) / portfolio_risk

    df_final = pd.concat([df_final, allocation])

df_final.sort_values("sharp_ratio", ascending=False)


,coke,googl,hd,ford,smg,jnj,ag,expected_return,portfolio_risk,sharp_ratio
0,0.000000,0.090909,0.012397,0.227273,0.388430,0.276860,0.004132,0.127764,0.183515,0.505485
0,0.010417,0.208333,0.003472,0.288194,0.312500,0.170139,0.006944,0.130921,0.191283,0.501464
0,0.000000,0.105769,0.009615,0.081731,0.336538,0.370192,0.096154,0.119883,0.172325,0.492575
0,0.042802,0.295720,0.000000,0.128405,0.315175,0.175097,0.042802,0.128796,0.190825,0.491529
0,0.036745,0.246719,0.002625,0.194226,0.254593,0.217848,0.047244,0.124791,0.183357,0.489706
...,...,...,...,...,...,...,...,...,...,...
0,0.197531,0.049383,0.500000,0.018519,0.179012,0.024691,0.030864,0.001223,0.417158,-0.080969
0,0.077844,0.119760,0.443114,0.017964,0.005988,0.233533,0.101796,0.003988,0.357281,-0.086799
0,0.025641,0.012821,0.538462,0.294872,0.044872,0.044872,0.038462,-0.006208,0.428921,-0.096074
0,0.311111,0.038889,0.500000,0.050000,0.005556,0.061111,0.033333,-0.010548,0.444603,-0.102447
